In [ ]:
import numpy as np
import deepSI
from deepSI import System_data
from deepSI import load_system_data
from deepSI import System_data_list
out= np.load('training-data.npz')
th_train = out['th']
u_train = out['u']

#data = np.load('test-simulation-submission-file.npz')
#u_test = data['u']
#th_test = data['th']
#print(u_train)





system_data = System_data(u=u_train,y=th_train)

#print(system_data.nu)
#system_data = system_data.to_IO_data(na=2,nb=3)
#system_data2 = load_system_data('training-data.npz')


if False: #train model
    sys = deepSI.fit_systems.SS_encoder(nx=13,na=2,nb=3) #load the encoder system
    sys.n_hidden_layers = 1
    sys.n_nodes_per_layer = 20
    sys.fit(system_data, val_sys_data=system_data[-12_000:], epochs=500, batch_size=1024, loss_kwargs=dict(nf=40)) 
    sys.save_system('Model_encoder\\encoder')

    
    sim_val = system_data[-25_000:] #larger validation set
    sys.bestfit = sys.apply_experiment(sim_val).NRMS(sim_val) #reset checkpoint
    #use all the data for final local minima search
    sys.fit(system_data,sim_val, epochs=200 ,batch_size=1024, loss_kwargs=dict(nf=40)) 
    sys.save_system('Model_encoder\\encoder_stepped')
    
else: #load model from file
    sys = deepSI.load_system('Model_encoder\encoder_final')

prediction = sys.one_step_ahead(system_data)

system_data_list=system_data.u
system_data_list = System_data_list(system_data_list)
print(system_data_list)
#
print(system_data)
prediction_data = system_data.to_IO_data(na=2,nb=3)
prediction_data2 = system_data.to_hist_future_data(na=2,nb=3,nf=1)
prediction_data3= system_data
prediction = sys.one_step_ahead(system_data_list)

#simulation = sys.apply_experiment(system_data)

print('train prediction errors:')
print('RMS:', prediction.RMS(system_data),'radians')
print('RMS:', prediction.RMS(system_data)/(2*np.pi)*360,'degrees')
print('NRMS:', prediction.NRMS(system_data),'%')
